In [2]:
import pandas as pd
import numpy as np
import scipy as sp
from scipy import stats
import boto3

In [3]:
s3 = boto3.resource('s3')
bucket = s3.Bucket('cnn-di-datsci-crosswalk-crosswalk-dev')
prefix_objs = bucket.objects.filter(Prefix = "KWU/TEST/HP_AUDIO_DW") # your path

bucket_name = 's3://cnn-di-datsci-crosswalk-crosswalk-dev/'

df = pd.DataFrame()

for obj in prefix_objs:
    path = bucket_name + obj.key
    prefix_df = pd.read_json(
        path,
        lines=True,
        compression='gzip',
        dtype = {'EX_GROUP': str, 'FEATURE': str, 'FEATURE_TYPE': str, 'VALUE': int, 'VISITOR_ID': str}
    )
    
    df = pd.concat([df, prefix_df])

In [28]:
# initialize
out = pd.DataFrame({'ex_group': pd.Series(dtype = 'str'),
                    'feature': pd.Series(dtype='str'),
                    'control': pd.Series(dtype='float'),
                    'variant': pd.Series(dtype='float'),
                    'change': pd.Series(dtype='float'),
                    'significant': pd.Series(dtype='str'),
                    'p_value': pd.Series(dtype='float')})

# queue
control = 'control'
variants = df[df['EX_GROUP'] != 'control']['EX_GROUP'].unique().tolist()
features = df['FEATURE'].unique().tolist()

for feature in features:
    
    for variant in variants:
  
        # feature value
        x = df[df['FEATURE'] == feature]
        feature_type = x['FEATURE_TYPE'].unique().tolist()

        variant_values = x[x['EX_GROUP'] == variant]['VALUE']
        control_values = x[x['EX_GROUP'] == control]['VALUE']

        # group average
        variant_stat = np.nanmean(variant_values)
        control_stat = np.nanmean(control_values)
      
   
        # nominal feature
        if feature_type == 'nominal':
  
            # compare
            change = ((variant_stat - control_stat) / control_stat)
            values = x.filter((x.EX_GROUP == variant) | (x.EX_GROUP == control)).toPandas()
            contigency = pd.crosstab(values['EX_GROUP'], values['VALUE'])
            stat, p, dof, ex = stats.chi2_contingency(contigency)
            sig = p <= 0.05

            # result
            row = pd.Series([variant, feature, control_stat, variant_stat, change, sig, p], index = out.columns)
            out = out.append(row, ignore_index=True)
    
        # continuous feature
        else:
      
            # compare
            change = ((variant_stat - control_stat) / control_stat)
            t, p = stats.ttest_ind(control_values, variant_values)
            sig = p <= 0.05
      
            # result
            row = pd.Series([variant, feature, control_stat, variant_stat, change, sig, p], index = out.columns)
            out = out.append(row, ignore_index=True)


print(out)

   ex_group                feature    control    variant    change  \
0   variant               pageview  12.547955  12.457546 -0.007205   
1   variant       pageview_article   4.104796   4.049635 -0.013438   
2   variant      pageview_homepage   6.386974   6.380561 -0.001004   
3   variant     pageview_videoleaf   0.630884   0.616865 -0.022221   
4   variant     pageview_livestory   0.087559   0.089400  0.021027   
5   variant             videostart   3.095340   2.954452 -0.045516   
6   variant     videostart_article   1.761502   1.645241 -0.066001   
7   variant    videostart_homepage   0.000314   0.000576  0.835879   
8   variant   videostart_videoleaf   0.445964   0.429560 -0.036785   
9   variant   videostart_livestory   0.046056   0.043936 -0.046040   
10  variant             audiostart   0.001102   0.005205  3.723779   
11  variant  timespent_per_session   2.889143   2.811703 -0.026804   
12  variant     timespent_homepage   2.995812   3.016499  0.006905   

   significant     

In [30]:
file_name = 'HP_AUDIO_DW_RESULTS.csv'
out.to_csv(file_name)

# s3.meta.client.upload_file(file_name, 'cnn-di-datsci-crosswalk-crosswalk-dev', 'KWU/TEST/HP_AUDIO_DW_RESULTS.csv')